# "Is there life after graduate school?"
> "An introduction to Dash with applications to doctoral degree recipients' job outlook."

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2
- use_plotly: true

Dash (https://dash.plotly.com/introduction) is a Python package that allows for easy app creation.  It works with plotly, the package that was used in my last blog post, to allow for simple coding of interactive visualizations.  For this dashboard, data from the NSF on doctorate recipients in the United States was used (https://ncses.nsf.gov/pubs/nsf19301/data).  

To get started, I downloaded Anaconda.  This allows me to run a local version of Python to get the app running.  Within Anaconda, I opened up Jupyter notebooks to code this dashboard.  After doing some standard data cleaning using the pandas package, I began to write the code for the web page.  My webpage contains four images in a two-by-two grid with drop down menus for two and radio items for the other two to allow for customization of the graphs and easier interpretation. 

To create the layout, dash bootstrap components were used (Read more about dash bootstrap location components here: https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/).  This package allows for the specification of size and location of each feature of the dashboard.  Rows must be specified first, and columns can be specified within each row. 

Within the row and column commands, html (Read more about html components here: https://dash.plotly.com/dash-html-components.  I only used Div, H1, and H2.) and dash core components (Read more about core components here: https://dash.plotly.com/dash-core-components.) were used to include text, graphs, dropdowns, and radio items.  I used html components to include text headers; H1 is called for the largest header, and H2 is called for the next largest header.  From dcc, dash core components, I created by interactive plots.  

To allow for interaction between the dropdowns and radio items and their corresponding plots, an app callback must be used.  This callback takes inputs from each dropdown and radio item and it outputs the appropriate plots.  Within the callback is where I utilized the plotly express code.  

The code is included below.

In [1]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])

NameError: name 'dash' is not defined

In [ ]:
app.layout = html.Div([
    dbc.Row(
        dbc.Col(
            html.H1("Is there life after graduate school?", style={'text-align': 'center'})
        )
    ),
    
    dbc.Row(
        dbc.Col(
            html.H2("What percentage of doctorate recipients have definite job commitments?", style={'text-align': 'center'})
        )
    ),
             
    dbc.Row([
        dbc.Col(dcc.Graph(id='field_graph', figure={}),
                width={'order': 1}),
    
        dbc.Col(dcc.Graph(id='dem_graph', figure={}),
                width={'order': 2})
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Dropdown(id="fields",
                              options=[
                                  {"label": "All fields", "value": 'All fields'},
                                  {"label": "Life Sciences", "value": 'Life sciences'},
                                  {"label": "Physical Sciences and Earth Sciences", "value":"Physical sciences and earth sciences"},
                                  {"label": "Mathematics and Computer Sciences","value": "Mathematics and computer sciences"},
                                  {"label": "Psychology and Social Sciences","value": "Psychology and social sciences"},
                                  {"label": "Engineering","value": "Engineering"},
                                  {"label": "Education","value": "Education"},
                                  {"label": "Humanities and Arts","value": "Humanities and arts"},
                                  {"label": "Other","value": "Other"},
                              ],
                              value=['All fields',
                                     'Life sciences',
                                    "Physical sciences and earth sciences",
                                    "Mathematics and computer sciences",
                                   "Psychology and social sciences",
                                    "Engineering",
                                    "Education",
                                    "Humanities and arts",
                                    "Other"
                                   ],
                             multi = True
                             ),
                width={'order': 1}
               ),
        
        dbc.Col(dcc.RadioItems(id="dem",
                               options=[
                                   {"label": "Sex   ", "value": 'sex'},
                                   {"label": "Race   ", "value": 'race'},
                                   {"label": "Immigration Status", "value":"cit"}],
                               value='sex',
                               style={'text-align': 'center'}
                              )
                ,
                width={'order': 2}
               )
    ]),
    
    dbc.Row(
        dbc.Col(
            html.H2("Where do doctorate recipients have definite job commitments?", style={'text-align': 'center'})
        )
    ),
        
    dbc.Row([
        dbc.Col(dcc.Graph(id='job_graph', figure={}),
               width = {'order': 1}),
        dbc.Col(dcc.Graph(id='dem2_graph', figure={}),
               width = {'order': 2}),
        
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Dropdown(id="job",
                 options=[
                     {"label": "All fields", "value": 'Total'},
                     {"label": "Life Sciences", "value": 'Life sciences'},
                     {"label": "Physical Sciences and Earth Sciences", "value":"Physical sciences and earth sciences"},
                     {"label": "Mathematics and Computer Sciences","value": "Mathematics and computer sciences"},
                     {"label": "Psychology and Social Sciences","value": "Psychology and social sciences"},
                     {"label": "Engineering","value": "Engineering"},
                     {"label": "Education","value": "Education"},
                     {"label": "Humanities and Arts","value": "Humanities and arts"}],
                 value='Total',
                 style={'text-align': 'center'}
                 )),
        
        dbc.Col(dcc.RadioItems(id="dem2",
                               options=[
                                   {"label": "Male   ", "value": 'Male'},
                                   {"label": "Female   ", "value": 'Female'}],
                               value='Male',
                               style={'text-align': 'center'}
                              )
                ,
                width={'order': 2}
               )
    ]),
    

])

In [ ]:
@app.callback(
    [Output(component_id='field_graph', component_property='figure'),
     Output(component_id='dem_graph', component_property='figure'),
    Output(component_id='job_graph', component_property='figure'),
    Output(component_id='dem2_graph', component_property='figure')],
    [Input(component_id='fields', component_property='value'),
     Input(component_id='dem', component_property='value'),
    Input(component_id='job', component_property='value'),
    Input(component_id='dem2', component_property='value')]
)

def update_graph(optf, opt, optf2, opt2):
    """Update graphs based on checklist and radio items."""
    
    df_field = byfield_long.copy()
    df_field = df_field[df_field['variable'].isin(optf)]
    
    df_job = ebf2_long[ebf2_long['variable'] == optf2]
    
    df = bycat_long.copy()
    if opt == 'sex':
        df = df[df['variable'].isin(['Male', 'Female'])]
    elif opt == 'race': 
        df = df[df['variable'].isin(['Hispanic or Latino',
                                             'American Indian or Alaska Native',
                                            'Black or African American',
                                            'White',
                                            'More than one race',
                                            'Other race or race not reported'])]
    else:
        df = df[df['variable'].isin(["U.S. citizen or permanent resident",
                                    "Temporary visa holder"])]
        
    df2 = ebd2_long[ebd2_long['variable'] == opt2]

        
    # Plotly Express
    figfield = px.line(
        df_field, x = 'Year', y = 'value', color = 'variable', 
        labels = {
            'value':'Percent with Definite Commitment',
            'variable':'Field'},
        title = "By Field"
    )
    
    fig = px.line(
        df, x = 'Year', y = 'value', color = 'variable', 
        labels = {
            'value':'Percent with Definite Commitment',
            'variable':'Demographic'},
        title = "By Demographic: Sex, Race, or Immigration Status"
    )
    
    figjob = px.area(df_job, 
                x = "Year",
                y = "value",
              color = "job",
              title = "By Field"
                    )
    
    fig2 = px.area(df2, 
                x = "Year",
                y = "value",
              color = "job",
              title = "By Sex: Male or Female"
                    )
  
    return figfield, fig, figjob, fig2

In [ ]:
if __name__ == '__main__': 
    app.run_server(debug = False)

The plots look at the data on definite job commitments for doctoral degree recipients in the United States by field of study and various demographic categories.  By field, Humanities and Arts have the lowest percentage of definite job commitments.  

Additionally, males have a higher percent commitment by about 2% every year measured than females and a similar difference is seen in US citizens having a higher percent commitment than visa holders.  After 2002, the overall percentage with definite job commitments decreased until 2012.  This could be due to the recession or many other factors.  

It's clear in the area plots on the bottom of the dashboard that the most common area of employment of doctoral degree recipients is academia.  There is a large difference of type of job commitment between fields, with fields like physical/earth science and engineering having a large percentage going into industry or business.  Other fields such as humanities and arts and education have a larger percentage going into academia.  

There are sex differences observed in job sector, with women more likely to enter academia than men and men entering industry of business more than women.  

If you run the Jupyter file through Anaconda on your local machine, the dashboard can be seen at this link: http://127.0.0.1:8050/.

So, is there life after graduate school?  Take a look for yourself at your field of study and other identifiers to see what the trends have been in the past.  